# Analysis of the results

In [4]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000

In [5]:
dfs = {}
for i in ["05", "10", "15"]:
    df = pd.read_csv("stats-{:s}.csv".format(i))
    df["instance"] = df.apply(lambda row: row["instance"].split(".")[0], axis=1)
    # rename ILP
    df["solver"] = df.apply(lambda row: "ILP" if row["solver"] == "ILP Solver (global):predictor (Python)" else row["solver"], axis=1)
    # merge to separate columns by instance
    bap = df[df["solver"] == "BAP"]            
    ilp = df[df["solver"] == "ILP"]
    df = pd.merge(bap, ilp, on="instance",suffixes=["_bap", "_ilp"])
    
    dfs[i] = df

In [6]:
for size in ["05", "10", "15"]:
    print()
    print(size)
    
    df = dfs[size]
    
    print("Overall performance:")
    for method in ["ilp", "bap"]:
        avg = df["time_" + method].mean() / 1000 # to s
        std = df["time_" + method].std() / 1000
        avg_nodes = df["metadata_bap"]
        print("time {:s} ${:4.2f} \pm {:4.2f}$".format(method, avg, std))    
                    
    # ILP was faster
    df_cur = df[df["time_ilp"] < df["time_bap"]]
    print(" - ilp was faster {:d} times".format(df_cur["instance"].count()))                
    for method in ["ilp", "bap"]:
        avg = df_cur["time_" + method].mean() / 1000
        std = df_cur["time_" + method].std() / 1000
        print("   time {:s} ${:4.2f} \\pm {:4.2f}$".format(method, avg, std))    
    bap_nodes_avg = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).mean()
    bap_nodes_min = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).min()
    bap_nodes_max = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).max()
    print("   bap nodes avg", bap_nodes_avg, "min", bap_nodes_min, "max", bap_nodes_max)
        
    
    df_cur = df[df["time_ilp"] > df["time_bap"]]
    print(" - bap was faster {:d} times".format(df_cur["instance"].count()))    
    for method in ["ilp", "bap"]:
        avg = df_cur["time_" + method].mean() / 1000
        std = df_cur["time_" + method].std() / 1000
        print("   time {:s} ${:4.2f} \\pm {:4.2f}$".format(method, avg, std))    
    bap_nodes_avg = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).mean()
    bap_nodes_min = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).min()
    bap_nodes_max = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).max()
    print("   bap nodes avg", bap_nodes_avg, "min", bap_nodes_min, "max", bap_nodes_max)


05
Overall performance:
time ilp $0.03 \pm 0.04$
time bap $0.14 \pm 0.21$
 - ilp was faster 80 times
   time ilp $0.02 \pm 0.02$
   time bap $0.16 \pm 0.23$
   bap nodes avg 3.5625 min 0 max 43
 - bap was faster 19 times
   time ilp $0.09 \pm 0.04$
   time bap $0.06 \pm 0.03$
   bap nodes avg 0.8947368421052632 min 0 max 1

10
Overall performance:
time ilp $1.79 \pm 4.70$
time bap $3.21 \pm 7.00$
 - ilp was faster 67 times
   time ilp $0.37 \pm 0.50$
   time bap $4.30 \pm 8.26$
   bap nodes avg 23.597014925373134 min 0 max 308
 - bap was faster 32 times
   time ilp $4.77 \pm 7.47$
   time bap $0.94 \pm 1.28$
   bap nodes avg 5.6875 min 1 max 74

15
Overall performance:
time ilp $45.37 \pm 113.48$
time bap $59.26 \pm 127.41$
 - ilp was faster 60 times
   time ilp $4.57 \pm 8.02$
   time bap $94.22 \pm 155.11$
   bap nodes avg 397.81666666666666 min 1 max 3254
 - bap was faster 40 times
   time ilp $106.57 \pm 161.82$
   time bap $6.82 \pm 10.51$
   bap nodes avg 22.975 min 1 max 300
